In [1]:
import torch
from torch import  nn


## 3.1.1 卷积层
- 计算过程
    - 书上的是滑动step后每个元素进行相乘
    - pytorch中是非常简单的


In [2]:
# 快速搭建卷积层
conv = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,stride=1,padding=1,dilation=1,groups=1,bias=True)  

# 查看卷积核的基本信息，本质上是一个Module
conv


Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [4]:
# 通过.weight和.bias查看卷积核的权重和偏置
conv.weight

Parameter containing:
tensor([[[[ 0.0377,  0.2636, -0.2682],
          [-0.1179,  0.3013,  0.2442],
          [ 0.3137, -0.0469, -0.1935]]]], requires_grad=True)

## 3.1.2 激活层函数
- 只有卷积层可以吗？
    - 答案是不行
        - 原因：神经网络如果仅仅是由线性的卷积运算堆叠组成，则其无法形成复杂的表达空间，也就很难提取出高语义的信息，因此还需要加入非线性的映射，又称为激活函数，可以逼近任意的非线性函数，以提升整个神经网络的表达能力。
- 在检测任务中常用的激活函数有：
    - Sigmoid
    - ReLU （为了缓解因为链式求导带来的梯度消失现象，所以才有ReLU，其加强版为：Leaky ReLU函数）
    - Softmax （多物体类别常用）
        

In [9]:
import torch
from torch import nn
input = torch.ones(1,1,2,2)
input
sigmod = nn.Sigmoid()
sigmod(input)

# 其余函数代码比较简单，暂时掠过


tensor([[[[0.7311, 0.7311],
          [0.7311, 0.7311]]]])

## 3.1.3 池化层
- 目的：降低特征图的参数量，从而提升计算速度，一种降采样操作(优化采样方法)
- 常见的池化层：
    - 最大值池化（Max Pooling）
    - 平均值池化（Average Polling）

In [11]:
import torch
from torch import nn

max_pooling = nn.MaxPool2d(2,stride=2)
aver_pooling = nn.AvgPool2d(2,stride=2)
input = torch.randn(1,1,4,4)
input

tensor([[[[-1.6962,  1.2758, -0.5516,  0.8100],
          [-1.3772,  2.0731, -0.8452, -0.8731],
          [ 0.3330, -1.8309, -0.7143, -2.6112],
          [ 0.6964,  0.8424,  0.6429,  0.3556]]]])

In [17]:
after_maxPolling = max_pooling(input)
after_maxPolling
# after_maxPolling.shape # 看到应用了max_pooling 后的张量形状都发生了改变，平均值也是一样


tensor([[[[2.0731, 0.8100],
          [0.8424, 0.6429]]]])

## 3.1.4 Dropout层 （一般被广泛应用到全连接层中，一般保留概率设置为0.5）
- 出现原因：
    - 参数过多然后样本又少，这样下去会产生过拟合
- 为什么可以防止过拟合？
    - 多模型平均
    - 减少神经元依赖
    - 生物进化
        

In [ ]:
import torch
from torch import nn

# PyTorch将元素置0来实现Dropout层，第一个参数为置0概率，第二个为是否原地操作
dropout = nn.Dropout(0.5,inplace=False)
input = torch.randn(2,64,7,7)
output = dropout(input)
output

## 3.1.5 BN层
- 出现原因:
    - 浅层参数的微弱变化经过多层线性变换和激活函数后会被放大，会造成深层的网络需要不断的调整以适应这些分布变化，最红导致模型难以训练和收敛
- BN层对每一个batch的输入特征进行白化操作：即去均值方差的过程：
    - $ 均值\mu = \frac{1}{m}  \sum_{1}^{m}x_{i} $
    - $ 方差\sigma_{B} ^{2}   = \frac{1}{m}  \sum_{1}^{m} (x_{i} - \mu)^{2} $
    - $白化 = \frac{x_{i}-\mu _{B} }{\sqrt{\sigma_{B}^{}2+\varepsilon  } }  $ 